In [52]:
import argparse
import datetime
import os
import math
import numpy as nps
import pandas as pd
import sys
from keras.utils import plot_model
from sklearn.preprocessing import scale
from timeit import default_timer as timer
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


In [53]:
### set paramter values
#------------------------------------------------------------------------
# general
#------------------------------------------------------------------------
training_ratio = 0.9            # ratio of training data to overall data
input_dim = 520
output_dim = 13                 # number of labels
verbose = 1                     # 0 for turning off logging
seed = 7                        # random number seed for reproducibility
### global constant variables
#------------------------------------------------------------------------
# general
#------------------------------------------------------------------------
INPUT_DIM = 520                 #  number of APs
VERBOSE = 1                     # 0 for turning off logging
#------------------------------------------------------------------------
# stacked auto encoder (sae)
#------------------------------------------------------------------------
# SAE_ACTIVATION = 'tanh'
SAE_ACTIVATION = 'relu'
SAE_BIAS = False
SAE_OPTIMIZER = 'adam'
SAE_LOSS = 'mse'
#------------------------------------------------------------------------
# classifier
#------------------------------------------------------------------------
CLASSIFIER_ACTIVATION = 'relu'
CLASSIFIER_BIAS = False
CLASSIFIER_OPTIMIZER = 'adam'
CLASSIFIER_LOSS = 'binary_crossentropy'
#------------------------------------------------------------------------
# input files
#------------------------------------------------------------------------
path_train = '../data/UJIIndoorLoc/trainingData2.csv'           # '-110' for the lack of AP.
path_validation = '../data/UJIIndoorLoc/validationData2.csv'    # ditto
#------------------------------------------------------------------------
# output files
#------------------------------------------------------------------------
path_base = '../my_results/'
path_out =  path_base + ''
#path_sae_model = path_base + '_sae_model.hdf5'

batch_size = 10
epochs = 20
#sae_hidden_layers = [256,128,64,128,256]
sae_hidden_layers = [64, 64, 256, 512]
#classifier_hidden_layers = [128,128]
classifier_hidden_layers =  [64,512,128]
dropout = 0.2
N = 8
scaling= 0.2

random_seed = 0

In [54]:
### initialize random seed generator of numpy
import random as rn

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(random_seed)
rn.seed(12345)



In [55]:
import tensorflow as tf

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(random_seed)  # initialize random seed generator of tensorflow
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

from keras.layers import Dense, Dropout
from keras.models import Sequential, load_model

In [56]:
train_df = pd.read_csv(path_train, header=0) # pass header=0 to be able to replace existing names
test_df = pd.read_csv(path_validation, header=0)

In [57]:
train_AP_features = scale(np.asarray(train_df.iloc[:,0:520]).astype(float), axis=1)

In [58]:
train_AP_features

array([[-0.17125017, -0.17125017, -0.17125017, ..., -0.17125017,
        -0.17125017, -0.17125017],
       [-0.16059846, -0.16059846, -0.16059846, ..., -0.16059846,
        -0.16059846, -0.16059846],
       [-0.15523113, -0.15523113, -0.15523113, ..., -0.15523113,
        -0.15523113, -0.15523113],
       ...,
       [-0.1077911 , -0.1077911 , -0.1077911 , ..., -0.1077911 ,
        -0.1077911 , -0.1077911 ],
       [-0.17141826, -0.17141826, -0.17141826, ..., -0.17141826,
        -0.17141826, -0.17141826],
       [-0.17331788, -0.17331788, -0.17331788, ..., -0.17331788,
        -0.17331788, -0.17331788]])

In [59]:
 # add a new column
train_df['REFPOINT'] = train_df.apply(lambda row: str(int(row['SPACEID'])) + str(int(row['RELATIVEPOSITION'])), axis=1)

In [60]:
train_df

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,REFPOINT
0,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7541.264300,4.864921e+06,2,1,106,2,2,23,1371713733,1062
1,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7536.621200,4.864934e+06,2,1,106,2,2,23,1371713691,1062
2,-110,-110,-110,-110,-110,-110,-110,-97,-110,-110,...,-7519.152400,4.864950e+06,2,1,103,2,2,23,1371714095,1032
3,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7524.570400,4.864934e+06,2,1,102,2,2,23,1371713807,1022
4,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7632.143600,4.864982e+06,0,0,122,2,11,13,1369909710,1222
5,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7533.896200,4.864939e+06,2,1,105,2,2,23,1371713841,1052
6,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7519.152400,4.864950e+06,2,1,103,2,2,23,1371713883,1032
7,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7527.451100,4.864929e+06,2,1,101,2,2,23,1371713775,1012
8,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7559.497300,4.864888e+06,2,1,112,2,2,23,1371714307,1122
9,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-7510.437173,4.864949e+06,2,1,103,1,2,23,1371714128,1031


In [61]:
blds = np.unique(train_df[['BUILDINGID']])
flrs = np.unique(train_df[['FLOOR']])

In [62]:
flrs

array([0, 1, 2, 3, 4], dtype=int64)

In [63]:
x_avg = {}
y_avg = {}
for bld in blds:
    for flr in flrs:
        # map reference points to sequential IDs per building-floor before building labels
        cond = (train_df['BUILDINGID']==bld) & (train_df['FLOOR']==flr)
        
        _, idx = np.unique(train_df.loc[cond, 'REFPOINT'], return_inverse=True) # refer to numpy.unique manual
        train_df.loc[cond, 'REFPOINT'] = idx
            
        # calculate the average coordinates of each building/floor
        x_avg[str(bld) + '-' + str(flr)] = np.mean(train_df.loc[cond, 'LONGITUDE'])
        y_avg[str(bld) + '-' + str(flr)] = np.mean(train_df.loc[cond, 'LATITUDE'])

In [64]:
len_train = len(train_df) 
len_train

19937

In [65]:
# for consistency in one-hot encoding
blds_all = np.asarray(pd.get_dummies(pd.concat([train_df['BUILDINGID'], test_df['BUILDINGID']])))
flrs_all = np.asarray(pd.get_dummies(pd.concat([train_df['FLOOR'], test_df['FLOOR']]))) # ditto

blds = blds_all[:len_train]
flrs = flrs_all[:len_train]

In [66]:
rfps = np.asarray(pd.get_dummies(train_df['REFPOINT']))
train_labels = np.concatenate((blds, flrs, train_df['LONGITUDE'][:,None],train_df['LATITUDE'][:,None]), axis=1)
OUTPUT_DIM = train_labels.shape[1]

In [67]:
OUTPUT_DIM

10

In [68]:
# split the training set into training and validation sets; 

# we will use the validation set at a testing set.
train_val_split = np.full((len(train_AP_features)), True)
train_val_split[int(len(train_AP_features)*training_ratio):len(train_AP_features)*99] = False

In [69]:
train_val_split

array([ True,  True,  True, ..., False, False, False])

In [70]:

x_train = train_AP_features[train_val_split]
y_train = train_labels[train_val_split]
x_val = train_AP_features[~train_val_split]
y_val = train_labels[~train_val_split]

In [71]:
y_train.shape

(17943, 10)

In [72]:
# create a model based on stacked autoencoder (SAE)
model = Sequential()
model.add(Dense(sae_hidden_layers[0], input_dim=INPUT_DIM, activation=SAE_ACTIVATION, use_bias=SAE_BIAS))
for units in sae_hidden_layers[1:]:
    model.add(Dense(units, activation=SAE_ACTIVATION, use_bias=SAE_BIAS))  
model.add(Dense(INPUT_DIM, activation=SAE_ACTIVATION, use_bias=SAE_BIAS))
model.compile(optimizer=SAE_OPTIMIZER, loss=SAE_LOSS)

# train the model
model.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, verbose=VERBOSE,shuffle=False)




Epoch 1/20
17943/17943 [==============================] - 11s 614us/step - loss: 0.8595
Epoch 2/20
17943/17943 [==============================] - 8s 433us/step - loss: 0.7979
Epoch 3/20
17943/17943 [==============================] - 8s 419us/step - loss: 0.7659
Epoch 4/20
17943/17943 [==============================] - 8s 418us/step - loss: 0.7423
Epoch 5/20
17943/17943 [==============================] - 7s 413us/step - loss: 0.7156
Epoch 6/20
17943/17943 [==============================] - 8s 443us/step - loss: 0.7019
Epoch 7/20
17943/17943 [==============================] - 8s 419us/step - loss: 0.6946
Epoch 8/20
17943/17943 [==============================] - 7s 410us/step - loss: 0.6881
Epoch 9/20
17943/17943 [==============================] - 8s 421us/step - loss: 0.6741
Epoch 10/20
17943/17943 [==============================] - 8s 424us/step - loss: 0.6628
Epoch 11/20
17943/17943 [==============================] - 8s 422us/step - loss: 0.6581
Epoch 12/20
17943/17943 [===============

In [87]:
# remove the decoder part
num_to_remove = (len(sae_hidden_layers) + 1) // 2
for i in range(num_to_remove):
    model.pop()
    
### build and train a complete model with the trained SAE encoder and a new classifier
model.add(Dropout(dropout))
for units in classifier_hidden_layers:
    model.add(Dense(units, activation=CLASSIFIER_ACTIVATION, use_bias=CLASSIFIER_BIAS))
    model.add(Dropout(dropout))
model.add(Dense(OUTPUT_DIM, activation='sigmoid', use_bias=CLASSIFIER_BIAS)) # 'sigmoid' for multi-label classification
#model.add(Dense(OUTPUT_DIM, use_bias=CLASSIFIER_BIAS)) # 'sigmoid' for multi-label classification
model.compile(optimizer=CLASSIFIER_OPTIMIZER, loss=CLASSIFIER_LOSS, metrics=['accuracy'])


In [88]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs, verbose=VERBOSE,shuffle=False)

Train on 17943 samples, validate on 1994 samples
Epoch 1/20
17943/17943 [==============================] - 13s 747us/step - loss: -7084968.7724 - acc: 0.5159 - val_loss: -6334170.9127 - val_acc: 0.5265
Epoch 2/20
17943/17943 [==============================] - 13s 699us/step - loss: -7027702.4441 - acc: 0.5523 - val_loss: -7140590.6443 - val_acc: 0.4752
Epoch 3/20
17943/17943 [==============================] - 13s 702us/step - loss: -6974659.9407 - acc: 0.5367 - val_loss: -6801665.3334 - val_acc: 0.5234
Epoch 4/20
17943/17943 [==============================] - 13s 743us/step - loss: -6868592.5980 - acc: 0.5505 - val_loss: -6797769.7492 - val_acc: 0.5234
Epoch 5/20
17943/17943 [==============================] - 13s 704us/step - loss: -6818372.1211 - acc: 0.5510 - val_loss: -6708167.3156 - val_acc: 0.5243
Epoch 6/20
17943/17943 [==============================] - 13s 703us/step - loss: -7309214.6981 - acc: 0.5495 - val_loss: -7312003.3363 - val_acc: 0.5185
Epoch 7/20
17943/17943 [=========

In [89]:
y_train.shape

(17943, 10)

In [90]:
# turn the given validation set into a testing set
test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]).astype(float), axis=1) # convert integer to float and scale jointly (axis=1)
x_test_utm = np.asarray(test_df['LONGITUDE'])
y_test_utm = np.asarray(test_df['LATITUDE'])
blds = blds_all[len_train:]
flrs = flrs_all[len_train:]


In [91]:
### evaluate the model
# calculate the accuracy of building and floor estimation
preds = model.predict(test_AP_features, batch_size=batch_size)
n_preds = preds.shape[0]

In [92]:
preds


array([[1. , 0. , 0. , ..., 0. , 0. , 1. ],
       [1. , 0. , 0. , ..., 0. , 0. , 1. ],
       [1. , 0. , 0. , ..., 0. , 0. , 1. ],
       ...,
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [1. , 0. , 0. , ..., 0. , 0. , 1. ],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]], dtype=float32)

In [303]:
# blds_results = (np.equal(np.argmax(test_labels[:, :3], axis=1), np.argmax(preds[:, :3], axis=1))).astype(int)
blds_results = (np.equal(np.argmax(blds, axis=1), np.argmax(preds[:, :3], axis=1))).astype(int)
acc_bld = blds_results.mean()

In [304]:
flrs_results = (np.equal(np.argmax(flrs, axis=1), np.argmax(preds[:, 3:8], axis=1))).astype(int)
acc_flr = flrs_results.mean()
acc_bf = (blds_results*flrs_results).mean()

In [305]:
acc_bf

0.8658865886588659

In [306]:
# calculate positioning error when building and floor are correctly estimated
mask = np.logical_and(blds_results, flrs_results) # mask index array for correct location of building and floor

In [307]:
x_test_utm = x_test_utm[mask]
y_test_utm = y_test_utm[mask]
blds = blds[mask]
flrs = flrs[mask]
rfps = (preds[mask])[:, 8:118]

In [308]:
preds

array([[1.0297575e-02, 9.9521565e-01, 2.1730443e-03, ..., 1.1972478e-04,
        3.2447293e-05, 1.2540325e-05],
       [2.5909934e-02, 1.7458746e-02, 9.6731722e-01, ..., 2.2604458e-04,
        3.8167986e-05, 1.2509167e-04],
       [1.7051795e-03, 2.0629088e-05, 9.9964464e-01, ..., 1.9397373e-10,
        1.2104942e-11, 7.6571575e-11],
       ...,
       [9.9999928e-01, 3.3589333e-06, 6.7075649e-07, ..., 3.2562130e-30,
        2.0217059e-22, 5.3918166e-24],
       [9.9999428e-01, 1.3992766e-05, 8.0608215e-06, ..., 4.7821076e-28,
        2.3350263e-21, 5.0457453e-23],
       [9.9999952e-01, 2.8441034e-06, 3.3277522e-07, ..., 4.7932230e-31,
        7.6818764e-23, 1.4844686e-24]], dtype=float32)

In [309]:
# number of correct building and floor location
n_success = len(blds)   
n_success

962

In [310]:
n_loc_failure = 0
sum_pos_err = 0.0
sum_pos_err_weighted = 0.0
idxs = np.argpartition(rfps, -N)[:, -N:]  # (unsorted) indexes of up to N nearest neighbors
threshold = scaling*np.amax(rfps, axis=1)

In [311]:
idxs

array([[ 2, 27,  1, ..., 30, 29,  0],
       [11, 10,  4, ...,  6,  7,  5],
       [11,  7,  9, ...,  5,  8,  6],
       ...,
       [40, 47, 41, ..., 45, 44, 46],
       [40, 43, 41, ..., 46, 45, 44],
       [40, 47, 41, ..., 45, 44, 46]], dtype=int64)

In [312]:
np.argpartition(rfps, -N)

array([[ 73, 109, 108, ...,  30,  29,   0],
       [ 72, 109, 108, ...,   6,   7,   5],
       [ 62, 109, 108, ...,   5,   8,   6],
       ...,
       [ 52, 109, 108, ...,  45,  44,  46],
       [  1, 109, 108, ...,  46,  45,  44],
       [ 52, 109, 108, ...,  45,  44,  46]], dtype=int64)

In [313]:
np.argpartition(rfps, -N)[:, -N:]

array([[ 2, 27,  1, ..., 30, 29,  0],
       [11, 10,  4, ...,  6,  7,  5],
       [11,  7,  9, ...,  5,  8,  6],
       ...,
       [40, 47, 41, ..., 45, 44, 46],
       [40, 43, 41, ..., 46, 45, 44],
       [40, 47, 41, ..., 45, 44, 46]], dtype=int64)

In [314]:
rfps.shape

(962, 110)

In [315]:
for i in range(n_success):
    xs = []
    ys = []
    ws = []
    for j in idxs[i]:
        rfp = np.zeros(110)
        rfp[j] = 1
        rows = np.where((train_labels == np.concatenate((blds[i], flrs[i], rfp))).all(axis=1)) # tuple of row indexes
        if rows[0].size > 0:
            if rfps[i][j] >= threshold[i]:
                xs.append(train_df.loc[train_df.index[rows[0][0]], 'LONGITUDE'])
                ys.append(train_df.loc[train_df.index[rows[0][0]], 'LATITUDE'])
                ws.append(rfps[i][j])
    if len(xs) > 0:
        sum_pos_err += math.sqrt((np.mean(xs)-x_test_utm[i])**2 + (np.mean(ys)-y_test_utm[i])**2)
        sum_pos_err_weighted += math.sqrt((np.average(xs, weights=ws)-x_test_utm[i])**2 + (np.average(ys, weights=ws)-y_test_utm[i])**2)
    else:
        n_loc_failure += 1
        key = str(np.argmax(blds[i])) + '-' + str(np.argmax(flrs[i]))
        pos_err = math.sqrt((x_avg[key]-x_test_utm[i])**2 + (y_avg[key]-y_test_utm[i])**2)
        sum_pos_err += pos_err
        sum_pos_err_weighted += pos_err

In [316]:
 # mean_pos_err = sum_pos_err / (n_success - n_loc_failure)
mean_pos_err = sum_pos_err / n_success
# mean_pos_err_weighted = sum_pos_err_weighted / (n_success - n_loc_failure)
mean_pos_err_weighted = sum_pos_err_weighted / n_success
loc_failure = n_loc_failure / n_success # rate of location estimation failure given that building and floor are correctly located

In [317]:
### print out final results
now = datetime.datetime.now
path_out = "../my_results/"
#path_out += "_" + now.strftime("%Y%m%d-%H%M%S") + ".txt"
path_out += "[SAE" + str(sae_hidden_layers) + "] "
path_out += "[Class" +str(classifier_hidden_layers) +  "] "
path_out += "[DropOut" + str(dropout)+"] "
path_out += "[PE" + str(round(mean_pos_err,2))+"] "
path_out += "[PEW" + str(round(mean_pos_err_weighted,2))+"] "
path_out += ".txt"
f = open(path_out, 'w')
f.write("#+STARTUP: showall\n")  # unfold everything when opening
f.write("* System parameters\n")
f.write("  - Numpy random number seed: %d\n" % random_seed)
f.write("  - Ratio of training data to overall data: %.2f\n" % training_ratio)
f.write("  - Number of epochs: %d\n" % epochs)
f.write("  - Batch size: %d\n" % batch_size)
f.write("  - Number of neighbours: %d\n" % N)
f.write("  - Scaling factor for threshold: %.2f\n" % scaling)
f.write("  - SAE hidden layers: %d" % sae_hidden_layers[0])
for units in sae_hidden_layers[1:]:
    f.write("-%d" % units)
f.write("\n")
f.write("  - SAE activation: %s\n" % SAE_ACTIVATION)
f.write("  - SAE bias: %s\n" % SAE_BIAS)
f.write("  - SAE optimizer: %s\n" % SAE_OPTIMIZER)
f.write("  - SAE loss: %s\n" % SAE_LOSS)
f.write("  - Classifier hidden layers: ")
if classifier_hidden_layers == '':
    f.write("N/A\n")
else:
    f.write("%d" % classifier_hidden_layers[0])
    for units in classifier_hidden_layers[1:]:
        f.write("-%d" % units)
    f.write("\n")
    f.write("  - Classifier hidden layer activation: %s\n" % CLASSIFIER_ACTIVATION)
f.write("  - Classifier bias: %s\n" % CLASSIFIER_BIAS)
f.write("  - Classifier optimizer: %s\n" % CLASSIFIER_OPTIMIZER)
f.write("  - Classifier loss: %s\n" % CLASSIFIER_LOSS)
f.write("  - Classifier dropout rate: %.2f\n" % dropout)
# f.write("  - Classifier class weight for buildings: %.2f\n" % building_weight)
# f.write("  - Classifier class weight for floors: %.2f\n" % floor_weight)
f.write("* Performance\n")
f.write("  - Accuracy (building): %e\n" % acc_bld)
f.write("  - Accuracy (floor): %e\n" % acc_flr)
f.write("  - Accuracy (building-floor): %e\n" % acc_bf)
f.write("  - Location estimation failure rate (given the correct building/floor): %e\n" % loc_failure)
f.write("  - Positioning error (meter): %e\n" % mean_pos_err)
f.write("  - Positioning error (weighted; meter): %e\n" % mean_pos_err_weighted)
f.close()


In [318]:
mean_pos_err_weighted

9.834623264678163

In [319]:
#9.777095051648537
#9.570682739573801
#9.417571286309927